# [Fe/H] Metallicity Distribution

In [33]:
from multiprocessing import Pool
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic

In [34]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [35]:
figure_setup()
settings = Settings()

In [36]:
METALL_RANGE: tuple = (-3, 1.5)
N_BINS: int = 40

In [37]:
def read_data(simulation: str, of: str, to: str) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    pd.DataFrame
        A data frame with the properties.
    """

    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)
    s.add_metal_abundance(of, to)
    s.tag_in_situ_stars()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df["[Fe/H]"] = s.metal_abundance[f"{of}/{to}"][is_real_star & is_main_obj]
    df["RegionTag"] = s.region_tag[is_real_star & is_main_obj]
    df["IsInSitu"] = s.is_in_situ[is_real_star & is_main_obj].astype("bool")

    return df

In [38]:
simulations = [f"au{i}_or_l4_s127" for i in range(1, 31)]

In [39]:
for simulation in simulations:
    df = read_data(simulation=simulation, of="Fe", to='H')

    fig = plt.figure(figsize=(7.4, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.tick_params(which='both', direction="in")
        ax.set_xlim(METALL_RANGE)
        ax.set_ylim(0, 0.2)
        ax.set_xticks([-2, -1, 0, 1])
        ax.set_xlabel('[Fe/H] [dex]')
        ax.set_ylabel(r"$f_\star$")
        ax.grid(True, ls='-', lw=0.25, c="gainsboro")
        ax.label_outer()

    for idx, component in enumerate(settings.components):

        n_stars_comp = (df["RegionTag"] == idx).sum()

        hist, bin_edges = np.histogram(
            a=df["[Fe/H]"][df["RegionTag"] == idx],
            bins=N_BINS, range=METALL_RANGE)
        bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
        axs[idx].plot(
            bin_centers, hist / n_stars_comp,
            c=settings.component_colors[component], lw=1)

        hist, _ = np.histogram(
            a=df["[Fe/H]"][(df["RegionTag"] == idx) & (df["IsInSitu"] == 1)],
            bins=N_BINS, range=METALL_RANGE)
        axs[idx].plot(
            bin_centers, hist / n_stars_comp,
            c='k', lw=0.75, ls=(0, (3, 1.5, 1, 1.5)), label="In-Situ Stars")

        hist, _ = np.histogram(
            a=df["[Fe/H]"][(df["RegionTag"] == idx) & (df["IsInSitu"] == 0)],
            bins=N_BINS, range=METALL_RANGE)
        axs[idx].plot(
            bin_centers, hist / n_stars_comp,
            c='k', lw=0.75, ls=(0, (5, 1.5)), label="Ex-Situ Stars")

        axs[idx].text(x=0.05, y=0.95, size=8.0, ha="left", va="top",
                    s=settings.component_labels[settings.components[idx]],
                    transform=axs[idx].transAxes)

    axs[3].legend(loc="lower left", framealpha=0, fontsize=6,
                    bbox_to_anchor=(0.009, 0.65))

    axs[0].text(
        x=0.95, y=0.95, size=6.0,
        s=r"$\texttt{" + simulation.upper() + "}$",
        ha='right', va='top', transform=axs[0].transAxes)

    fig.savefig(
        f"../images/FeH_metallicity_distribution/{simulation}.pdf")
    plt.close(fig)